In [5]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import numpy as np

In [6]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

2021-10-03 01:49:35.929098: W external/org_tensorflow/tensorflow/compiler/xla/service/platform_util.cc:205] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 51050315776


[-0.372111    0.2642311  -0.18252774 -0.7368198  -0.44030386 -0.15214427
 -0.6713536  -0.59086424  0.73168874  0.56730247]


In [3]:
import jax
jax.devices()

[GpuDevice(id=0, process_index=0)]

In [4]:
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU


1.18 ms ± 38 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1e-2):
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

layer_sizes = [2, 3]
param_scale = 0.1
step_size = 0.01
num_epochs = 10
batch_size = 1
n_targets = 3
params = init_network_params(layer_sizes, random.PRNGKey(0))

In [6]:
params

[(DeviceArray([[-0.01454695,  0.00973438],
               [-0.00217348,  0.00691426],
               [-0.01011404,  0.00400959]], dtype=float32),
  DeviceArray([-0.00066073,  0.00166766,  0.01178003], dtype=float32))]

In [7]:
from jax.scipy.special import logsumexp

def relu(x):
    return jnp.maximum(0, x)

def q_value(params, x):
    # per-example predictions
    activations = x
    for w, b in params[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = relu(outputs)
  
    final_w, final_b = params[-1]
    logits = jnp.dot(final_w, activations) + final_b
    return logits - logsumexp(logits)

In [14]:
x = random.normal(random.PRNGKey(1), (3,))

int(jnp.argmax(x))

2

In [11]:
q_value(params, x)

DeviceArray([-1.0965854, -1.101934 , -1.0973257], dtype=float32)

In [3]:
import numpy as np
np.argmax(np.random.random(3))

0

In [20]:
# def one_hot(x, k, dtype=jnp.float32):
#     """Create a one-hot encoding of x of size k."""
#     return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
# def accuracy(params, images, targets):
#     target_class = jnp.argmax(targets, axis=1)
#     predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
#     return jnp.mean(predicted_class == target_class)


#         delta = learning*(reward + 
#                          discount*np.max(Q[state2_adj[0], 
#                                            state2_adj[1]]) - 
#                          Q[state_adj[0], state_adj[1],action])
#         Q[state_adj[0], state_adj[1],action] += delta

def loss(params, next_state, reward, cur_state_rewards, action, discount, learning):
    future_rewards = q_value(params, jnp.array(next_state[0], next_state[1]))
    delta = jnp.array([0,0,0])
    delta[action] = learning*(reward + discount*jnp.max(future_rewards) - 
                                 cut_state_rewards[action])
    return delta

def update(params, next_state, reward, cur_state_rewards, action, discount, learning):
    grads = grad(loss)(params, next_state, reward, cur_state_rewards, action, discount, learning)
    return [(w - step_size * dw, b - step_size * db)
        for (w, b), (dw, db) in zip(params, grads)]

def terminal_state_loss(reward, action, cur_state_rewards):
    delta = jnp.array([0,0,0])
    delta[action] = reward - cur_state_rewards[action]
    return delta
    
def update_terminal_state(params, reward, action, cur_state_rewards):
    grads = grad(terminal_state_loss)(reward, action, cur_state_rewards)
    return [(w - step_size * dw, b - step_size * db)
        for (w, b), (dw, db) in zip(params, grads)]

In [21]:
while done != True:   
#     # Render environment for last five episodes
#     if i >= (episodes - 20):
#         env.render()

    # In any case, get the rewards for current state
    cur_state_rewards = q_value(params, jnp.array([cur_state[0], cur_state[1]]))
    
    # Determine next action - epsilon greedy strategy
    if np.random.random() < 1 - epsilon:
        action = np.argmax(cur_state_rewards) 
    else:
        action = np.random.randint(0, env.action_space.n)

    # Get next state and reward
    next_state, reward, done, info = env.step(action) 

    #Allow for terminal states
    if done and next_state[0] >= 0.5:
        
        params = update_terminal_state(params, reward, action, cur_state_rewards)
        
        ## UPDATE NN based on q_value

    # Adjust Q value for current state
    else:
        params = update(params, next_state, reward, cur_state_rewards, action, discount, learning)

    # Update variables
    tot_reward += reward
    cur_state = next_state

NameError: name 'done' is not defined

In [2]:
import numpy as np
import gym
import matplotlib.pyplot as plt

# Import and initialize Mountain Car Environment
env = gym.make('CartPole-v0')
env.reset()

array([-0.01700249, -0.03022148, -0.03244098,  0.02547737], dtype=float32)

In [3]:
print(env.observation_space.low, env.observation_space.low)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38] [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [5]:
print(env.action_space)

Discrete(3)


In [7]:
# Try to apply Deep Q networks for this problem

In [8]:
env.step(2)

(array([-0.5664178 ,  0.00133015], dtype=float32), -1.0, False, {})

In [9]:
env.render()

True

In [2]:
from gym import wrappers
env = wrappers.Monitor(env, "./gym-results", force=True)

/home/sudhakars/anaconda3/envs/gym/lib/python3.9/site-packages/gym/wrappers/monitor.py:31: UserWarning: The Monitor wrapper is being deprecated in favor of gym.wrappers.RecordVideo and gym.wrappers.RecordEpisodeStatistics (see https://github.com/openai/gym/issues/2297)
  warnings.warn(


In [5]:
env.render()

True

In [13]:
!pip install tensorflow-datasets

     |████████████████████████████████| 4.0 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 4.9 MB/s eta 0:00:011
     |████████████████████████████████| 86 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 6.4 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=b600d1911cfc051bd4fb5e18d08e5997a26229c43b1d9e8fb8fde4d68b1c8f0e
  Stored in directory: /home/sudhakars/.cache/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=f133545c7e377e3491b7a502a87b77486911c3557d9782a90da62c20ecc5ad81
  Stored in directory: /home/sudhakars/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a93795

In [18]:
def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = batched_predict(params, images)
  return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]

In [41]:
import tensorflow as tf

In [16]:
import tensorflow as tf
# Ensure TF does not see GPU and grab all GPU memory.
tf.config.set_visible_devices([], device_type='GPU')

import tensorflow_datasets as tfds

data_dir = '/tmp/tfds'

# Fetch full datasets for evaluation
# tfds.load returns tf.Tensors (or tf.data.Datasets if batch_size != -1)
# You can convert them to NumPy arrays (or iterables of NumPy arrays) with tfds.dataset_as_numpy
mnist_data, info = tfds.load(name="mnist", batch_size=-1, data_dir=data_dir, with_info=True)
mnist_data = tfds.as_numpy(mnist_data)
train_data, test_data = mnist_data['train'], mnist_data['test']
num_labels = info.features['label'].num_classes
h, w, c = info.features['image'].shape
num_pixels = h * w * c

# Full train set
train_images, train_labels = train_data['image'], train_data['label']
train_images = jnp.reshape(train_images, (len(train_images), num_pixels))
train_labels = one_hot(train_labels, num_labels)

# Full test set
test_images, test_labels = test_data['image'], test_data['label']
test_images = jnp.reshape(test_images, (len(test_images), num_pixels))
test_labels = one_hot(test_labels, num_labels)

In [8]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

layer_sizes = [784, 512, 512, 10]
param_scale = 0.1
step_size = 0.01
num_epochs = 10
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.PRNGKey(0))

In [9]:
x = np.random.randn(batch_size, 784)

In [11]:
q_value(params, x)

TypeError: Incompatible shapes for dot: got (512, 784) and (128, 784).

In [17]:
import time

def get_train_batches():
  # as_supervised=True gives us the (image, label) as a tuple instead of a dict
  ds = tfds.load(name='mnist', split='train', as_supervised=True, data_dir=data_dir)
  # You can build up an arbitrary tf.data input pipeline
  ds = ds.batch(batch_size).prefetch(1)
  # tfds.dataset_as_numpy converts the tf.data.Dataset into an iterable of NumPy arrays
  return tfds.as_numpy(ds)

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in get_train_batches():
    x = jnp.reshape(x, (len(x), num_pixels))
    y = one_hot(y, num_labels)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

2021-10-03 00:43:20.907532: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


NameError: name 'update' is not defined